In [1]:
# | default_exp _multi_agents.planning_team

In [2]:
# | export

from typing import List, Any, Dict, Union

from contextlib import contextmanager
import random
import os
import autogen
import openai
import json
from dotenv import load_dotenv
from unittest.mock import Mock

In [3]:
# | export

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

api_key_sweeden = os.getenv("AZURE_OPENAI_API_KEY_SWEEDEN")
api_base_sweeden = "https://airt-openai-sweden.openai.azure.com/"

api_key_canada = os.getenv("AZURE_OPENAI_API_KEY_CANADA")
api_base_canada = "https://airt-openai-canada.openai.azure.com/"

In [4]:
# | export

openai.api_type = "azure"
    
openai.api_version = "2023-07-01-preview"

CONFIG_LIST = [
    {
        "model": "gpt-4",
        "api_key": api_key_sweeden,
        "api_base": api_base_sweeden,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
    },{
        "model": "gpt-4",
        "api_key": api_key_canada,
        "api_base": api_base_canada,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-canada-gpt4"

    },
#     {
#         "model": "gpt-3.5-turbo-16k",
#         "api_key": api_key_sweeden,
#         "api_base": api_base_sweeden,
#         "api_type": openai.api_type,
#         "api_version": openai.api_version,
#         "engine": "airt-gpt35-turbo-16k",
#     }
]

assert openai.api_key is not None, "Please set your AZURE_OPENAI_API_KEY environment variable."

In [5]:
planning_team_input = "{\"Roles\": [{\"Name\": \"PYTHON_DEV_GPT\", \"Description\": \"a professional Python developer AI that assists in writing Python scripts and functions\"}, {\"Name\": \"QA_GPT\", \"Description\": \"a professional Quality Assurance AI that assists in testing Python scripts and functions for bugs and errors\"}, {\"Name\": \"DOC_WRITER_GPT\", \"Description\": \"a professional Documentation Writer AI that assists in writing clear and concise documentation for Python scripts and functions\"}], \"Goals\": [\"Write a Python function that checks if a string is a palindrome, ignoring spaces, punctuation, and capitalization.\", \"Test the function thoroughly to ensure it works as expected.\", \"Write clear and concise documentation for the function.\"]}"

planning_team_input_dict = json.loads(planning_team_input)
planning_team_input_dict

{'Roles': [{'Name': 'PYTHON_DEV_GPT',
   'Description': 'a professional Python developer AI that assists in writing Python scripts and functions'},
  {'Name': 'QA_GPT',
   'Description': 'a professional Quality Assurance AI that assists in testing Python scripts and functions for bugs and errors'},
  {'Name': 'DOC_WRITER_GPT',
   'Description': 'a professional Documentation Writer AI that assists in writing clear and concise documentation for Python scripts and functions'}],
 'Goals': ['Write a Python function that checks if a string is a palindrome, ignoring spaces, punctuation, and capitalization.',
  'Test the function thoroughly to ensure it works as expected.',
  'Write clear and concise documentation for the function.']}

In [6]:
# | export

create_execution_team_config = {
    "name": "create_execution_team",
    "description": "Create the team which will execute the plan",
    "parameters": {
        "type": "object",
        "properties": {
            "plan": {
                "type": "string",
                "description": "The plan for the execution team",
            },
            "roles": {
                "type": "string",
                "description": "List of the team roles in charge of executing the plan (e.g. roles=['QA', 'Developer', 'Systrem_Arhitect'])",
            }
        },
        "required": ["plan, roles"],
    },
}

ask_for_additional_info_config = {
    "name": "ask_for_additional_info",
    "description": "Ask for the additional information from the user",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Question for the user",
            },
        },
        "required": ["question"],
    },
}

answer_to_execution_team_config = {
    "name": "answer_to_execution_team",
    "description": "Answer to the question which was asked by the execution team",
    "parameters": {
        "type": "object",
        "properties": {
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            },
        },
        "required": ["answer"],
    },
}

llm_config = {
    "config_list": CONFIG_LIST,
    "seed": 42,
    "temperature": 0.2,
    "functions": [create_execution_team_config, ask_for_additional_info_config, answer_to_execution_team_config]
}

In [7]:
# | export

ask_for_additional_info = Mock()
ask_for_additional_info.side_effect = ["I'm not sure", "Do what ever you think is the best solution"] * 5

create_execution_team = Mock()
create_execution_team.side_effect = ["Should I log the errors?", "Completed, proceed with the next goal"] * 5

answer_to_execution_team = Mock()
answer_to_execution_team.side_effect = ["Do I need a kafka broker?", "What is the email of our client?", "whuch libraries should I use?",
                                        "Completed, proceed with the next goal"] * 5


function_map = {
    "create_execution_team": create_execution_team,
    "ask_for_additional_info": ask_for_additional_info,
    "answer_to_execution_team": answer_to_execution_team,
}

def is_termination_msg(x: str) -> bool:
    return "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE")

def create_member(name: str, description: str) -> autogen.Agent:
        name = name.lower().replace(" ", "_")
        system_message = f"""You are {name}, {description}
        
        Your task is to chat with other team mambers and try to solve the given task.
        Do NOT try to finish the task until other team members give their opinion.
        """
        print(f"{system_message}\n{'*'*100}")
        
        return autogen.AssistantAgent(
            name=name,
            llm_config=llm_config,
            system_message=system_message,
            is_termination_msg=is_termination_msg,
            code_execution_config={"work_dir": "planning"},
            function_map=function_map
        )

In [8]:
# | export

def create_groupchat(planning_team_input_dict):
    members = []
    for role in planning_team_input_dict["Roles"]:
        members.append(create_member(role["Name"], role["Description"]))

    groupchat = autogen.GroupChat(agents=members, messages=[], max_round=60)
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        is_termination_msg=is_termination_msg,
    )
    return members, manager

In [9]:
goals = "\n".join([f"{i+1}. {goal}\n"for i, goal in enumerate(planning_team_input_dict["Goals"])])
print(goals)

1. Write a Python function that checks if a string is a palindrome, ignoring spaces, punctuation, and capitalization.

2. Test the function thoroughly to ensure it works as expected.

3. Write clear and concise documentation for the function.



In [10]:
# | export

def create_message(planning_team_input_dict, members):
#     goal = planning_team_input_dict["Goals"][0]
    goals = "\n".join([f"{i+1}. {goal}\n"for i, goal in enumerate(planning_team_input_dict["Goals"])])
    roles = ", ".join([str(member.name) for member in members])

    init_message = f"""
    You are a team dedicated for creating an effective plan for each of the following goals:
    \n{goals}


    The team is consisting of the following roles: {roles}.


    Play to your strengths as an LLM and pursue simple strategies with no legal complications.


    ## Guidelines
    1. Go through the list of goals one by one. Discuss about the current goal with other team members, create a detailed plan 
    for accomplishing the goal and create an execution team which will use the created plan for solving the current goal 
    2. Always create a plan only for the current goal
    3. If execution team asks a question, first anlyze the question with your team members 
       (DO NOT answer the question before consulting with your team).
       Once you know the answer to the question, send it to the execution team by suggesting the answer_to_execution_team function
    4. Once the execution team has completed the goal, move to the next one
    5. If you have some doubts ask the user for clarification by suggesting ask_for_additional_info function
    6. Write "TERMINATE" once all the goals have been completed


    ## Constraints
    You operate within the following constraints:
    1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
    2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
    3. You can ask and answer questions from other team members or suggest function listed below e.g. command_name


    ## Commands
    You have access to the following commands:
    1. list_files: Lists Files in a Directory, params: (directory: string)
    2. read_file: Read an existing file, params: (filename: string)
    3. create_execution_team: Creates the execution team, params: (plan: string, roles: List[string])
    4. ask_for_additional_info: Ask the user for additional information, params: (question: string)
    5. answer_to_execution_team: Answer the question asked by the execution team, params: (answer: string)


    ## Resources
    You can leverage access to the following resources:
    1. Long Term memory management.
    2. File output.
    3. Command execution


    ## Best practices
    1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
    2. Constructively self-criticize your big-picture behavior constantly.
    3. Reflect on past decisions and strategies to refine your approach.
    4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.
    5. If you have some doubts, ask question


    ## Goals
    For your task, you must create ONLY a PLAN for each of the following goals:
    \n{goals}
    """

    message = f"""
    {init_message}

    You should chat with other team members until you have the complete understandig of the goal.
    Once all the team members have understood the goal, determine exactly one command to use based on the given goal and the progress you have made so far,
    and respond by suggesting one of the previously defined Commands:
    """

    return message

In [11]:
# from unittest.mock import patch

# with patch("__main__.ask_for_additional_info", side_effect=["I'm not sure", "Do what ever you think is the best solution"]):
#     with patch("__main__.create_execution_team", side_effect=["Should I log the errors?", "Completed\nTERMINATE"]):
members, manager = create_groupchat(planning_team_input_dict)
message = create_message(planning_team_input_dict, members)
manager.initiate_chat(manager, message=message)



You are python_dev_gpt, a professional Python developer AI that assists in writing Python scripts and functions
        
        Your task is to chat with other team mambers and try to solve the given task.
        Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
You are qa_gpt, a professional Quality Assurance AI that assists in testing Python scripts and functions for bugs and errors
        
        Your task is to chat with other team mambers and try to solve the given task.
        Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
You are doc_writer_gpt, a professional Documentation Writer AI that assists in writing clear and concise documentation for Python scripts and functions
        
        Your task is to ch

KeyboardInterrupt: 

In [ ]:
"""
1. Write a Python function that checks if a string is a palindrome, ignoring spaces, punctuation, and capitalization.

2. Test the function thoroughly to ensure it works as expected.

3. Write clear and concise documentation for the function.
"""